In [1]:
import scipy.io
import numpy as np
import decimal
import pickle
from matplotlib import pyplot as plt
from matplotlib import rcParams

In [2]:
rcParams['figure.figsize'] = (20, 12)
rcParams.update({'font.size': 16})

Функция, которая считает Q(T) только для CO2

In [3]:
def do_Qt16(T):
    mol = 2
    Q_arr = np.zeros(10)
    for iso in range(1, 11):
        mol = str(mol)
        iso = str(iso)
        file = 'QTpy/'+mol+'_'+iso+'.QTpy'

        QTdict = {}
        with open(file, 'rb') as handle:
            QTdict = pickle.loads(handle.read())
        if(T==int(T)):
            key=str(int(T))
            Q = QTdict[key]
        else:
            key=str(int(T))
            Q1 = float(QTdict[key])
            key=str(int(T+1))
            Q2 = float(QTdict[key])
            QT = Q1+(Q2-Q1)*(T-int(T))
            Q = QT
        if(int(iso) < 10):
            Q_arr[int(iso)] = Q
        else:
            Q_arr[0] = Q
    return Q_arr

Функция, которая считает контур Фойгта вокруг точки w0

In [4]:
def contur_Humlicek(w, w0, wL0, wD):
    y = wL0/wD
    x = np.zeros(len(w))
    for i in range(len(w)):
        x[i] = (w[i]-w0)/wD
    t = np.zeros(len(w), dtype=np.complex_)
    for i in range(len(w)):
        t[i] = y - 1j*x[i]
    u = np.zeros(len(x), dtype=np.complex_)
    for i in range(len(x)):
        u[i] = t[i]*t[i]
    s = np.abs(x) + y    
    p = (1e-150)*np.ones(len(w))
    
    ff1 = np.where(s >= 15) # region 1 (s>=15)
    if len(ff1) != 0:
        w4 = t[ff1]*0.5641896/(0.5 + u[ff1])
        p[ff1] = np.real(w4)/(np.sqrt(np.pi)*wD)
    
    ff2 = np.where((s<15)&(s>=5.5)) # region 2 (5.5<=s<15)
    if len(ff2) != 0:
        w4 = t[ff2]*(1.410474+u[ff2]*0.5641896)/(0.75+u[ff2]*(3.0+u[ff2]))
        p[ff2] = np.real(w4)/(np.sqrt(np.pi)*wD)
    
    ff3 = np.where((s<5.5) & (y>=0.195*np.abs(x)-0.176)) # region 3 (s<5.5, y>=0.195*abs(x)-0.176)
    if len(ff3) != 0:
        w4 = (16.4955 + t[ff3]*20.20933 + u[ff3]*11.96482 + t[ff3]*u[ff3]*3.778987 + 
              u[ff3]*u[ff3]*0.5642236)/(16.4955 + t[ff3]*38.82363 + 
                                                              u[ff3]*39.27121 + t[ff3]*u[ff3]*21.69274 + 
                                                              u[ff3]*u[ff3]*6.699398 + t[ff3]*u[ff3]*u[ff3])
        p[ff3] = np.real(w4)/(np.sqrt(np.pi)*wD)
    
    ff4 = np.where((s<5.5) & (y<0.195*np.abs(x)-0.176)) # region 4 (s<5.5, y<0.195*abs(x)-0.176)
    if len(ff4) != 0:
        w4 = np.exp(u[ff4]) - t[ff4]*(36183.31 - u[ff4]*(3321.9905 - u[ff4]*(1540.787 - 
                                                                             u[ff4]*(219.0313 - u[ff4]*(35.76683 - 
                                                                                                        u[ff4]*(1.320522 - u[ff4]*0.56419))))))/(32066.6 - u[ff4]*(24322.84 - 
                                                                                                                                        u[ff4]*(9022.228 - u[ff4]*(2186.181 - 
                                                                                                                                        u[ff4]*(364.2191 - u[ff4]*(61.57037 - 
                                                                                                                                        u[ff4]*(1.841439 - u[ff4])))))))
        p[ff4]= np.real(w4)/(np.sqrt(np.pi)*wD)
    return p

Функция, считывающая параметры молекулы

In [5]:
# считывание параметров молекулы
def read_molpar16(iso):
    if iso==1:
        isoN=626
        Sn=9.84204E-01
        Q296=2.8609E+02
        gj=1
        molM=43.989830
    elif iso==2:
        isoN=636
        Sn=1.10574E-02
        Q296=5.7664E+02
        gj=2
        molM=44.993185  
    elif iso==3:
        isoN=628
        Sn=3.94707E-03
        Q296=6.0781E+02
        gj=1
        molM=45.994076  
    elif iso==4:
        isoN=627
        Sn=7.33989E-04
        Q296=3.5426E+03
        gj=6
        molM=44.994045  
    elif iso==5:
        isoN=638
        Sn=4.43446E-05
        Q296=1.2255E+03
        gj=2
        molM=46.997431  
    elif iso==6:
        isoN=637
        Sn=8.24623E-06
        Q296=7.1413E+03
        gj=12 
        molM=45.997400
    elif iso==7:
        isoN=828
        Sn=3.95734E-06
        Q296=3.2342E+02
        gj=1
        molM=47.998322 
    elif iso==8:
        isoN=827
        Sn=1.47180E-06
        Q296=3.7666E+03
        gj=6
        molM=46.998291  
    elif iso==9:
        isoN=727
        Sn=1.36847E-07
        Q296=1.0972E+04
        gj=1
        molM=45.998262 
    elif iso==0:
        isoN=838
        Sn=4.44600E-08
        Q296=6.5224E+02
        gj=2
        molM=49.001675
    return isoN, Sn, Q296, gj, molM

Функция, считывающая параметры линии поглощения

In [6]:
def readline(file):
    data = scipy.io.loadmat(file) # файл с инфой про линии поглощения для нужного спектрального диапазона
    lines = np.ravel(data['lines'])[0]
    iso = np.ravel(lines[1]) # массив с номерами изотопов
    v = np.ravel(lines[2]) # массив волновых чисел
    S = np.ravel(lines[3]) # массив интенсивностей линий
    A = np.ravel(lines[4]) # массив коэффициентов Эйнштейна
    air = np.ravel(lines[5]) # массив HMHM(air)
    self = np.ravel(lines[6]) # массив HWHM(self)
    E = np.ravel(lines[7]) # массив энергий нижних состояний
    nt = np.ravel(lines[8]) # массив коэффициентов зависимостей air от T
    dlt = np.ravel(lines[9]) # массив сдвигов линий
    # удаляет все NaN
    ff = np.where(np.isnan(iso) == 0)
    iso = iso[ff]
    v = v[ff]
    S = S[ff]
    A = A[ff]
    air = air[ff]
    self = self[ff]
    E = E[ff]
    nt = nt[ff]
    dlt = dlt[ff]
    # ff -- это массив индексов, на которых не содержится NaN в массиве iso
    return iso, v, S, A, air, self, E, nt, dlt

Функция для подсчёта сечений

In [7]:
def cross(T, p, w1, w2, isn):
    # константы
    R = 8.31441 # универсальная газовая постоянная [Дж/(мол*К)]
    c = 299792458. # скорость света [м/с]
    c2 = 1.438775221 # вторая постоянная Планка [см*к] (c2=hc/kB)
    
    iso, v, S, A, air, self, E, nt, dlt = readline('02_hit16.mat')
    wstep = 0.001 # см^-1
    
    if p >= 1:
        wLim0 = 30. # расширение диапазона
    elif (p < 1)&(p >= 0.1):
        wLim0 = 25.
    elif (p < 0.1)&(p >= 0.01):
        wLim0 = 20.
    elif (p < 0.01)&(p >= 1e-04):
        wLim0 = 15.
    elif (p < 1e-04)&(p >= 1e-07):
        wLim0 = 10.
    elif p < 1e-07:
        wLim0 = 5.
            
    if isn == 100: # номер изотопа (100 - все изотопы)
    # фильтрация, оставляем только нужные нам волновые числа
        if p >= 0.1:
            fv = np.where((v >= w1 - wLim0) & (v <= w2 + wLim0))
        elif (p<0.1)&(p>=0.01):
            fv = np.where((v >= w1 - wLim0) & (v <= w2 + wLim0) & (S >= 1e-30))
        elif (p<0.01)&(p>=1e-03):
            fv = np.where((v >= w1 - wLim0) & (v <= w2 + wLim0) & (S >= 1e-29))
        elif (p<1e-03)&(p>=1e-04): 
            fv = np.where((v >= w1 - wLim0) & (v <= w2 + wLim0) & (S >= 1e-28))
        elif (p<1e-04)&(p>=1e-05): 
            fv = np.where((v >= w1 - wLim0) & (v <= w2 + wLim0) & (S >= 1e-27))
        elif (p<1e-05)&(p>=1e-06):
            fv = np.where((v >= w1 - wLim0) & (v <= w2 + wLim0) & (S >= 1e-26))
        elif p<1e-06:
            fv = np.where((v >= w1 - wLim0) & (v <= w2 + wLim0) & (S >= 1e-25))
        numiso = int(np.max(iso)) # кол-во изотопов
        QQT = np.zeros(numiso + 1) # это Q(T) для каждого изотопа
        Sn=np.zeros(numiso + 1) # это распространенность изотопа в атмосфере
        Q296=np.zeros(numiso + 1) # это Q(296 К) для каждого изотопа
        mu=np.zeros(numiso + 1) # это молярная масса каждого изотопа [г/моль]
        for i in range(numiso + 1): 
            isoN1, Sn1, Q296_1, gj1, mu1 = read_molpar16(i)
            Sn[i]=Sn1
            Q296[i]=Q296_1
            mu[i]=mu1
        QQT = do_Qt16(T)
    else:
        if p >= 0.1:
            fv = np.where((v >= w1 - wLim0) & (v <= w2 + wLim0) & (iso == isn))
        elif (p<0.1) & (p>=0.01):
            fv = np.where((v >= w1 - wLim0) & (v <= w2 + wLim0) & (S >= 1e-30) & (iso == isn))
        elif (p<0.01) & (p>=1e-03):
            fv = np.where((v >= w1 - wLim0) & (v <= w2 + wLim0) & (S >= 1e-29) & (iso == isn))
        elif (p<1e-03) & (p>=1e-04):
            fv = np.where((v >= w1 - wLim0) & (v <= w2 + wLim0) & (S >= 1e-28) & (iso == isn))
        elif (p<1e-04) & (p>=1e-05): 
            fv = np.where((v >= w1 - wLim0) & (v <= w2 + wLim0) & (S >= 1e-27) & (iso == isn))
        elif (p<1e-05) & (p>=1e-06):
            fv = np.where((v >= w1 - wLim0) & (v <= w2 + wLim0) & (S >= 1e-26) & (iso == isn))
        elif p<1e-06:
            fv = np.where((v >= w1 - wLim0) & (v <= w2 + wLim0) & (S >= 1e-25) & (iso == isn))
        QQT0_arr = do_Qt16(T)
        QQT0 = QQT0_arr[isn]
        isoN0, Sn0, Q0, gj0, mu0 = read_molpar16(isn)
            
    iso = iso[fv]
    v = v[fv]
    S = S[fv]
    A = A[fv]
    air = air[fv]
    self = self[fv]
    E = E[fv]
    nt = nt[fv]
    dlt = dlt[fv]
    nW = len(v) # кол-во волновых чисел
        
    # сетка
    wGas = np.arange(w1 - wLim0, w2 + wLim0, wstep)
    nG = len(wGas)  # размер сетки
    # если массив волновых чисел в итоге пуст, то cGas = ...
    if v.size == 0:
        cGas = (1.0e-150)*np.ones(nG)
        
    # сортировка массивов по возрастанию v
    ix = np.argsort(v)
    v = np.take(v, ix)
    iso = np.take(iso, ix)
    S = np.take(S, ix)
    A = np.take(A, ix)
    air = np.take(air, ix)
    self = np.take(self, ix)
    E = np.take(E, ix)
    nt = np.take(nt, ix)
    dlt = np.take(dlt, ix)
    # убираем повторение v(j)
    if nW > 1:
        # прогоняем один раз
        dv = v[1:len(v)] - v[0:len(v) - 1] # массив
        for i in range(len(dv)):
            if dv[i] < 5e-07:
                dv[i] = 5e-07
        for j in range(1, len(v)):
                v[j] = v[j - 1] + dv[j - 1]
        # прогоняем второй раз
        dv = v[1:len(v)] - v[0:len(v) - 1]
        for i in range(len(dv)):
            if dv[i] < 5e-08:
                dv[i] = 5e-08
        for j in range(1, len(v)):
            v[j] = v[j - 1] + dv[j - 1]
        
    wL = self*(p/1013.)*((296./T)**0.73) # уширение за счет столкновений [см^-1]
    cGas = (1.0e-150)*np.ones(nG)
    S0 = np.zeros(nW)
    for g in range(nW):
        if isn == 100:
            dQ = Q296[int(iso[g])]/QQT[int(iso[g])] # отношение Q296/Q(T)
            wD = np.sqrt(2*1000*R*T/mu[int(iso[g])])*v[g]/c # Доплеровское уширение
            Rn=1.0
        else:
            dQ = Q0/QQT0 # отношение Q296/Q(T)
            wD = np.sqrt(2*1000*R*T/mu0)*v[g]/c
            Rn = 1/Sn0 # нормировка
            
        # подсчёт S(T)
        St = Rn*S[g]*dQ*np.exp(c2*E[g]*((1/296.) - (1/T)))*(1 - np.exp(-c2*v[g]/T))/(1 - np.exp(-c2*v[g]/296.))
        if (St > 1e-26):
            wLim = wLim0
        else:
            wLim = 10.
        wFind = np.where((wGas >= v[g] - wLim) & (wGas <= v[g] + wLim))
        wLine = wGas[wFind]

        fV = (1.0e-150)*np.ones(nG)
        fV0 = contur_Humlicek(wLine, v[g], wL[g], wD)
        fV[wFind] = St*fV0
        cGas = cGas + fV
        
    cGas = cGas[np.where((wGas >= w1) & (wGas <= w2))]
    wGas = wGas[np.where((wGas >= w1) & (wGas <= w2))]
    return cGas, wGas

In [8]:
# профили T, p 
data1 = scipy.io.loadmat('pos12_SEC_Tp_co2_0-100km_3155-3220cm-1.mat')
T = np.ravel(data1['temp'])
P = np.ravel(data1['pres'])
alt = np.ravel(data1['alt'])

In [9]:
data_custom = dict.fromkeys(['SECmat_co2', 'alt', 'pres', 'temp', 'wGas'])
data_custom['alt'] = alt
data_custom['pres'] = P
data_custom['temp'] = T

In [10]:
for i in range(len(P)):
    cGas_, wGas = cross(T[i], P[i], 3200, 3500, 100)
    if (i == 0):
        cGas = np.empty(len(cGas_))
    # создание матрицы сечений
    cGas = np.row_stack((cGas, cGas_))
data_custom['wGas'] = wGas
data_custom['SECmat_co2'] = cGas
scipy.io.savemat('secmat_custom_v2.mat', data_custom)

In [11]:
data_custom

{'SECmat_co2': array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  5.80355793e-32,   5.73264478e-32,   5.66305796e-32, ...,
           3.46106153e-30,   2.01916972e-30,   2.19177801e-30],
        [  1.02792648e-31,   1.01536883e-31,   1.00304606e-31, ...,
           5.24448207e-30,   3.17429103e-30,   3.16294828e-30],
        ..., 
        [  2.34296745e-27,   2.31438471e-27,   2.28634861e-27, ...,
           3.87094394e-26,   3.82742590e-26,   3.81530824e-26],
        [  3.65563972e-27,   3.61111449e-27,   3.56747098e-27, ...,
           7.19405746e-26,   7.04289958e-26,   6.96817460e-26],
        [  5.22988386e-27,   5.16644923e-27,   5.10439386e-27, ...,
           1.37792463e-25,   1.33157235e-25,   1.30274664e-25]]),
 'alt': array([ 100.,   95.,   90.,   85.,   80.,   75.,   70.,   65.,   60.,
          55.,   50.,   45.,   40.,   35.,   30.,   25.,   20.,   15.,
          10.,    5.,    0.])